In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('11_final_cohort_alldata.csv.gz')
data.head()

# fact check

In [ ]:
# prompt: check if there's any subject_id that have record charttime after charttime of the row's with ad=1

import pandas as pd

# Assuming 'data' DataFrame is already loaded as in the previous code snippet.
# If not, load it first:
# data = pd.read_csv('11_final_cohort_alldata.csv.gz')

# Convert 'charttime' to datetime objects if it's not already
data['storetime'] = pd.to_datetime(data['storetime'])

# Find rows where ad=1
ad_rows = data[data['ad'] == 1]

# Check if any subject_id has a record with a later charttime than any of the ad=1 rows for the same subject_id.
for subject_id in data['subject_id'].unique():
  subject_data = data[data['subject_id'] == subject_id]
  ad_subject_data = ad_rows[ad_rows['subject_id'] == subject_id]
  if not ad_subject_data.empty:
    max_ad_charttime = ad_subject_data['storetime'].max()
    if any(subject_data['storetime'] > max_ad_charttime):
      print(f"Subject ID {subject_id} has records after the latest storetime of ad=1 rows.")



In [ ]:
len(data["subject_id"].unique())

In [ ]:
notes=pd.read_csv('10_final_cohort_notes.csv.gz')
notes.head()

In [ ]:
notes["note_type"].value_counts()

In [ ]:
# prompt: give me the all rows with one subject_id have at least one row AD=1

import pandas as pd

# Group by subject_id and check if any 'ad' value is 1 within each group
subject_ad_check = data.groupby('subject_id')['ad'].any()

# Get the subject_ids where at least one 'ad' value is 1
subject_ids_with_ad = subject_ad_check[subject_ad_check == True].index

# Filter the original DataFrame to include only rows where subject_id is in the list above
ad_df = data[data['subject_id'].isin(subject_ids_with_ad)]

ad_df

In [ ]:
person=ad_df[ad_df["subject_id"]==10002131].sort_values(["note_seq"])
person

In [ ]:
person.iloc[1,:]["text"]

In [ ]:
# prompt: give me an example of a subject_id with all rows ad=0 but case_status=1

# Assuming 'data' DataFrame is already loaded as in the previous code snippet.
# If not, load it first:
# data = pd.read_csv('11_final_cohort_alldata.csv.gz')

# Find subject_ids where all 'ad' values are 0 but 'case_status' is 1
result = data.groupby('subject_id').agg({'ad': 'max', 'case_status': 'max'})
target_subjects = result[(result['ad'] == 0) & (result['case_status'] == 1)].index.tolist()

# Print the subject IDs that meet the criteria
if target_subjects:
  print(f"Subject IDs with all 'ad' values equal to 0 and 'case_status' equal to 1:\n{target_subjects}")
else:
  print("No subjects found that meet the criteria.")


In [ ]:
person_nonote=data[data["subject_id"]==10348850].sort_values(["note_seq"])
person_nonote

In [ ]:
person_nonote.iloc[0,:]["text"]

In [ ]:
data = data.sort_values(by=['subject_id', 'note_seq']).reset_index(drop=True)

In [ ]:
# prompt: give me another person (person3) with AD in their first ever note

import pandas as pd

# Group by subject_id and get the first note for each subject
first_notes = data.groupby('subject_id').first().reset_index()

# Find the first subject with AD in their first note
person3 = first_notes[first_notes['ad'] == 1]

# Print the text of the first note for the third person with AD
print(person3)


In [ ]:
data[data["subject_id"]==10015785].iloc[0]["text"]

# Trim dataset

In [ ]:
import pandas as pd

# Assuming 'data' DataFrame is already loaded as in the previous code snippet.
# If not, load it first:
# data = pd.read_csv('11_final_cohort_alldata.csv.gz')

# Create the two dataframes based on 'case_status'
case_status_1 = data[data['case_status'] == 1]
case_status_0 = data[data['case_status'] == 0]

# Now you have two separate DataFrames:
# case_status_1: Contains rows where case_status is 1
# case_status_0: Contains rows where case_status is 0

# You can now work with these DataFrames individually.  For example, to display the first few rows:
print("DataFrame with case_status = 1:")
print(len(case_status_1["subject_id"].unique()))

print("\nDataFrame with case_status = 0:")
print(len(case_status_0["subject_id"].unique()))


In [ ]:
case_status_1_sorted = case_status_1.sort_values(by=['subject_id', 'note_seq']).reset_index(drop=True)

In [ ]:
# Group data by subject_id
grouped = case_status_1_sorted.groupby('subject_id')

# Function to filter rows within each group
def filter_rows(group):
  ad_indices = group[group['ad'] == 1].index
  if not ad_indices.empty:
    first_ad_index = ad_indices.min()
    return group.loc[:first_ad_index-1]
  else:
    return group

# Apply the function to each group
filtered_data = grouped.apply(filter_rows)

# Reset index if needed
filtered_data = filtered_data.reset_index(drop=True)

filtered_data

### check how many patients we have left

In [ ]:
len(filtered_data['subject_id'].unique())

In [ ]:
text_concat = filtered_data.groupby('subject_id')['text'].apply(lambda x: '\nNew Note:\n'.join(x)).rename('text').reset_index()

In [ ]:
text_concat

In [ ]:
text_concat.iloc[0]["text"]

# Build final dataset

In [ ]:
# Then, take the max of all other columns (excluding 'text')
other_max = data.drop(columns='text').groupby('subject_id').max().reset_index()

# Combine both into one DataFrame
df_grouped = pd.merge(other_max, text_concat, on='subject_id', how='left', indicator=True)

# View result
df_grouped

## double check

In [ ]:
len(df_grouped[df_grouped["_merge"]=="both"])

In [ ]:
len(df_grouped[df_grouped["case_status"]==1])

In [ ]:
len(df_grouped[df_grouped["case_status"]==0])

In [ ]:
df_grouped.to_csv("trimed_dataset_perid.csv")